In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
salaries= pd.read_csv('../data/salaries_data.csv')

test = pd.read_csv('../data/testeo.csv')

In [3]:
#Borramos columnas que no hace falta
test.drop('salary', inplace=True, axis=1)
test.drop('salary_currency', inplace=True, axis=1)

In [4]:
salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [5]:
def diction(col):
    lista = list(set(list(salaries[col].unique()) + list(test[col].unique())))
    nums =[i for i in range(len(lista))]
    print({(lista[i]): (nums[i]) for i in range(len(lista))})
    return {(lista[i]): (nums[i]) for i in range(len(lista))}

dictio_experience_level = diction('experience_level')
dictio_employment_type = diction('employment_type')
dictio_job_title = diction('job_title')
dictio_employee_residence = diction('employee_residence')
dictio_company_location = diction('company_location')
dictio_company_size = diction('company_size')

{'EN': 0, 'SE': 1, 'MI': 2, 'EX': 3}
{'FT': 0, 'CT': 1, 'PT': 2, 'FL': 3}
{'Lead Machine Learning Engineer': 0, 'Lead Data Scientist': 1, 'Data Analyst': 2, 'Data Architect': 3, 'Data Analytics Manager': 4, 'Principal Data Scientist': 5, 'Machine Learning Infrastructure Engineer': 6, 'Data Science Consultant': 7, 'Director of Data Engineering': 8, 'NLP Engineer': 9, 'Computer Vision Software Engineer': 10, 'Financial Data Analyst': 11, 'Product Data Analyst': 12, 'Head of Data Science': 13, 'Machine Learning Developer': 14, 'Research Scientist': 15, 'Finance Data Analyst': 16, 'Machine Learning Manager': 17, 'ETL Developer': 18, 'Business Data Analyst': 19, 'Data Engineering Manager': 20, 'Analytics Engineer': 21, '3D Computer Vision Researcher': 22, 'Data Science Manager': 23, 'Director of Data Science': 24, 'Head of Data': 25, 'Data Scientist': 26, 'Machine Learning Engineer': 27, 'Data Science Engineer': 28, 'Data Specialist': 29, 'Applied Data Scientist': 30, 'BI Data Analyst': 31,

In [6]:
salaries_copy_1 = salaries.copy(deep=True)
salaries_copy_1.head()

salaries_copy_1.drop(['salary', 'salary_currency'], inplace=True,axis=1)
print(salaries_copy_1.head())

test_copy_1 = test.copy(deep=True)
print(test_copy_1.head())

   work_year experience_level employment_type        job_title  salary_in_usd  \
0       2022               SE              FT    Data Engineer         140250   
1       2022               SE              FT    Data Engineer         135000   
2       2021               MI              FT  BI Data Analyst         100000   
3       2021               MI              CT      ML Engineer         270000   
4       2021               MI              FT    Data Engineer          26005   

  employee_residence  remote_ratio company_location company_size  
0                 US           100               US            M  
1                 US           100               US            M  
2                 US           100               US            M  
3                 US           100               US            L  
4                 RO             0               US            L  
   work_year experience_level employment_type                   job_title  \
0       2020               SE     

In [7]:
test_copy_1.shape, salaries_copy_1.shape

((107, 8), (500, 9))

In [8]:
test_copy_1['y'] = -1
test_copy_1.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y
0,2020,SE,FT,Machine Learning Scientist,JP,0,JP,S,-1
1,2020,MI,FT,Lead Data Analyst,US,100,US,L,-1
2,2020,MI,FT,Data Analyst,US,100,US,L,-1
3,2020,MI,FT,Machine Learning Engineer,CN,0,CN,M,-1
4,2020,MI,FT,Product Data Analyst,IN,100,IN,L,-1


In [9]:
test_copy_1.shape, salaries_copy_1.shape

((107, 9), (500, 9))

In [10]:
todo = pd.concat([test_copy_1, salaries_copy_1])

In [11]:
todo.job_title = todo.job_title.apply(lambda x: dictio_job_title[x])
todo.employment_type = todo.employment_type.apply(lambda x: dictio_employment_type[x])
todo.employee_residence = todo.employee_residence.apply(lambda x: dictio_employee_residence[x])
todo.experience_level = todo.experience_level.apply(lambda x: dictio_experience_level[x])
todo.company_size = todo.company_size.apply(lambda x: dictio_company_size[x])
todo.company_location = todo.company_location.apply(lambda x: dictio_company_location[x])
todo.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,1,0,34,22,0,18,2,-1.0,NaN
1,2020,2,0,41,5,100,5,0,-1.0,NaN
2,2020,2,0,2,5,100,5,0,-1.0,NaN
3,2020,2,0,27,39,0,34,1,-1.0,NaN
4,2020,2,0,12,54,100,47,0,-1.0,NaN


In [12]:
ok = todo.loc[todo['y'] == -1]
ok.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,1,0,34,22,0,18,2,-1.0,NaN
1,2020,2,0,41,5,100,5,0,-1.0,NaN
2,2020,2,0,2,5,100,5,0,-1.0,NaN
3,2020,2,0,27,39,0,34,1,-1.0,NaN
4,2020,2,0,12,54,100,47,0,-1.0,NaN


In [13]:
ok.shape

(107, 10)

In [14]:
ok_s = todo.loc[todo['y'] != -1]
ok_s.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2022,1,0,36,5,100,5,1,NaN,140250.0
1,2022,1,0,36,5,100,5,1,NaN,135000.0
2,2021,2,0,31,5,100,5,1,NaN,100000.0
3,2021,2,1,33,5,100,5,0,NaN,270000.0
4,2021,2,0,36,8,0,5,0,NaN,26005.0


In [15]:
ok_s.shape

(500, 10)

In [16]:
ok_s.drop('y', axis=1, inplace=True)
ok.drop('salary_in_usd', axis=1, inplace=True)
ok.drop('y', axis=1, inplace=True)

## Modelos

In [17]:
X = ok_s.drop('salary_in_usd', axis = 1)
y = ok_s.salary_in_usd

In [18]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400, 8), (100, 8), (400,), (100,))

In [19]:
#Inicializamos modelos
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
dtr=DTR()
lgbmr=LGBMR()

#Entrenamientos
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train, verbose=0)
dtr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)

#Predicciones
y_pred_rfr = rfr.predict(X_test)
y_pred_etr = etr.predict(X_test)
y_pred_gbr = gbr.predict(X_test)
y_pred_xgbr = xgbr.predict(X_test)
y_pred_ctr = ctr.predict(X_test)
y_pred_dtr = dtr.predict(X_test)
y_pred_lgbmr = lgbmr.predict(X_test)

#Calculo de error
print(mse(y_test, y_pred_rfr, squared=False), mse(y_test, y_pred_etr, squared=False), mse(y_test, y_pred_gbr, squared=False), 
mse(y_test, y_pred_xgbr, squared=False), mse(y_test, y_pred_ctr, squared=False), mse(y_test, y_pred_dtr, squared=False),
mse(y_test, y_pred_lgbmr, squared=False))

54006.72106513985 60734.57884141029 50592.35345341891 54939.04367421978 52557.32278582287 68697.18785922742 51329.94973770452


### Mejor Modelo: LightGBMR

In [20]:
lgbmr.fit(X_train, y_train)

y_pred_lgbmr = lgbmr.predict(ok)
y_pred_lgbmr[:10]

len(y_pred_lgbmr)

107

## Exportar

In [21]:
res = pd.read_csv('../data/muestra.csv')
res.salary_in_usd = y_pred_lgbmr
res.head()

,id,salary_in_usd
0,0,107208.864773
1,1,174047.182679
2,2,125077.679379
3,3,32419.216365
4,4,79881.607002


In [22]:
res.to_csv('../data/resultado_lgbmr_contodo.csv', index = False)